In [1]:
import pandas as pd
import numpy as np
import math
import mpmath

In [5]:
#politicas fixas
rewards1 = pd.read_csv('/Users/dulpneto/workspace/OpenAI-GYM-CarRacing-DQN-Risk/r_bound_300/rewards_2.csv')

#politica encontrada
rewards2 = pd.read_csv('/Users/dulpneto/workspace/OpenAI-GYM-CarRacing-DQN-Risk/r_bound_300_v2/rewards_r_bound.csv')
#rewards = pd.read_csv('/Users/dulpneto/workspace/OpenAI-GYM-CarRacing-DQN-Risk/rewards_tiles.csv')

#rewards = pd.concat([rewards2], axis=0, ignore_index=True)
rewards = pd.concat([rewards1, rewards2], axis=0, ignore_index=True)


lambs = []
for i in range(-8,3):
    lambs.append(i)
    
#for i in range(0,11):
#    lambs.append(round(0.7726278200644+(i/100000000000000), 14))


for lamb in lambs:
    if lamb == 0:
        rewards['{}'.format(lamb)] = rewards.apply(lambda row : row[1], axis = 1)
        continue
    try:
        rewards['{}'.format(lamb)] = rewards.apply(lambda row : np.sign(lamb) * np.exp(lamb * row[1]), axis = 1)
    except (ValueError, OverflowError) as error:
        print(error)
        print('Error {}'.format(lamb))

rewards

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: overflow encountered in exp


,policy,reward,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2
0,1,-39.4,-7.755695e+136,-6.003688e+119,-4.647459e+102,-3.597601e+85,-2.784905e+68,-2.155796e+51,-1.668803e+34,-1.291822e+17,-39.4,7.741006e-18,5.992318e-35
1,1,-59.3,-1.069799e+206,-1.886425e+180,-3.326417e+154,-5.865619e+128,-1.034311e+103,-1.823847e+77,-3.216070e+51,-5.671041e+25,-59.3,1.763345e-26,3.109385e-52
2,1,-200.3,-inf,-inf,-inf,-inf,-inf,-9.280132e+260,-9.514138e+173,-9.754044e+86,-200.3,1.025216e-87,1.051067e-174
3,1,-116.6,-inf,-inf,-6.798601e+303,-1.562007e+253,-3.588776e+202,-8.245363e+151,-1.894407e+101,-4.352478e+50,-116.6,2.297542e-51,5.278698e-102
4,1,-57.4,-2.679329e+199,-3.158805e+174,-3.724085e+149,-4.390524e+124,-5.176224e+99,-6.102528e+74,-7.194598e+49,-8.482098e+24,-57.4,1.178954e-25,1.389932e-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5493,4-p,-39.1,-7.035807e+135,-7.351902e+118,-7.682197e+101,-8.027332e+84,-8.387972e+67,-8.764814e+50,-9.158587e+33,-9.570051e+16,-39.1,1.044927e-17,1.091871e-34
5494,4-p,-40.0,-9.423977e+138,-4.003639e+121,-1.700888e+104,-7.225974e+86,-3.069850e+69,-1.304181e+52,-5.540622e+34,-2.353853e+17,-40.0,4.248354e-18,1.804851e-35
5495,4-p,-40.3,-1.038822e+140,-3.269440e+122,-1.028977e+105,-3.238457e+87,-1.019226e+70,-3.207767e+52,-1.009567e+35,-3.177369e+17,-40.3,3.147258e-18,9.905234e-36
5496,4-p,-43.9,-3.343659e+152,-2.875374e+133,-2.472672e+114,-2.126370e+95,-1.828568e+76,-1.572473e+57,-1.352245e+38,-1.162861e+19,-43.9,8.599481e-20,7.395107e-39


In [6]:
rewards_by_policy = rewards.groupby('policy')
rewards_by_policy.agg({'reward': ['mean', 'var', 'min', 'max']})

reward                          
             mean          var    min   max
policy                                     
1      -52.625900  1748.718558 -343.7 -16.5
2      -47.389900  1189.025514 -275.4 -16.6
3      -21.184400    22.250527  -92.7 -19.6
4      -19.914800     0.147569  -21.2 -19.0
5      -20.500000     0.000000  -20.5 -20.5
0-p    -56.519192    26.139118  -73.2 -47.8
1-p    -71.764000    23.086570  -79.2 -59.1
2-p    -27.056000    19.475014  -52.3 -21.8
3-p    -22.937374    12.013793  -44.6 -19.8
4-p    -34.087000     9.808617  -56.3 -30.5

In [7]:
agg = {}
for lamb in lambs:
    agg['{}'.format(lamb)] = ['mean']
        
def highlight_max(s):
    is_max = s == s.max()
    return ['background: lightgreen' if cell else '' for cell in is_max]        
rewards_by_policy_formated = rewards_by_policy.agg(agg).style.format('{:.2e}').highlight_max(color='lightgreen')
rewards_by_policy_formated
  

,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
policy,,,,,,,,,,,
1,-inf,-inf,-inf,-inf,-inf,-inf,-3.42e+295,-1.87e+146,-5.26e+01,3.71e-09,9.25e-17
2,-inf,-inf,-inf,-inf,-inf,-inf,-1.62e+236,-4.15e+116,-4.74e+01,2.22e-09,3.47e-17
3,-inf,-6.51e+278,-3.59e+238,-1.97e+198,-1.09e+158,-5.99e+117,-3.30e+77,-1.82e+37,-2.12e+01,1.29e-09,1.96e-18
4,-1.96e+71,-1.59e+62,-1.39e+53,-1.33e+44,-1.45e+35,-1.83e+26,-2.72e+17,-4.81e+08,-1.99e+01,2.41e-09,6.59e-18
5,-1.68e+71,-2.10e+62,-2.62e+53,-3.27e+44,-4.09e+35,-5.12e+26,-6.40e+17,-8.00e+08,-2.05e+01,1.25e-09,1.56e-18
0-p,-2.12e+252,-3.44e+220,-5.58e+188,-9.04e+156,-1.46e+125,-2.37e+93,-3.85e+61,-6.55e+29,-5.65e+01,4.22e-23,3.62e-44
1-p,-3.02e+273,-1.23e+239,-5.04e+204,-2.09e+170,-8.91e+135,-4.00e+101,-2.04e+67,-1.54e+33,-7.18e+01,4.08e-28,5.62e-54
2-p,-5.11e+179,-9.89e+156,-1.91e+134,-3.70e+111,-7.15e+88,-1.38e+66,-2.67e+43,-5.17e+20,-2.71e+01,3.81e-11,7.20e-21


In [13]:
for act_index in range(5):
    print(act_index)

0
1
2
3
4
